# Apllication C2translate sur modèle base whisper fine tuned français

In [ ]:
https://huggingface.co/brandenkmurray/faster-whisper-large-v3-french-distil-dec16

In [ ]:
pip install ctranslate2 transformers

git lfs install
git clone https://huggingface.co/BrunoHays/whisper-large-v3-french-illuin




In [ ]:
sudo apt install ffmpeg libavcodec-dev libavformat-dev libavdevice-dev libavutil-dev

# Download large distilled 16 + ctranslate2

In [2]:
python -c "from huggingface_hub import snapshot_download; snapshot_download(repo_id='bofenghuang/whisper-large-v3-french-distil-dec16', local_dir='./models/whisper-large-v3-french-distil-dec16', allow_patterns='ctranslate2/*')"


SyntaxError: invalid syntax (2141624470.py, line 1)

# Download large distilled 16

In [ ]:
from huggingface_hub import snapshot_download

# Télécharger tous les fichiers du modèle PyTorch natif
snapshot_download(
    repo_id='bofenghuang/whisper-large-v3-french-distil-dec16',
    local_dir='./models/whisper-large-v3-french-distil-dec16',
    allow_patterns='*'   # Télécharge tout le repo
)


In [2]:
from datasets import load_dataset
from faster_whisper import WhisperModel

# Load model
model = WhisperModel("./models/whisper-large-v3-french-distil-dec16/ctranslate2", device="cpu", compute_type="int8")  # Run on GPU with FP16

# Example audio
dataset = load_dataset("bofenghuang/asr-dummy", "fr", split="test")
sample = dataset[0]["audio"]["array"].astype("float32")

segments, info = model.transcribe(sample, beam_size=5, language="fr")

for segment in segments:
    print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))


[0.00s -> 5.52s] Nous allons construire des radiotélescopes géants, comme celui-ci qui est mon préféré.


# Modèle Whisper large distilled 32 -> 16 + Ctranslate2

# Int 8

In [5]:
from faster_whisper import WhisperModel

# Chemin vers le modèle CTranslate2
model_path = "./models/whisper-large-v3-french-distil-dec16-ct2/ctranslate2"

# Instanciation du modèle sur CPU avec quantification int8
model = WhisperModel(
    model_path,
    device="cpu",        # CPU uniquement
    compute_type="int8"  # accélération quantifiée
)

# Transcription d'un fichier audio
audio_file = "test_20201111_17_21_20_8.wav"
segments, info = model.transcribe(
    audio_file,
    beam_size=5,
    language="fr",
    condition_on_previous_text=False
)

# Affichage de la langue détectée
print("Langue détectée :", info.language)

# Affichage du texte segment par segment
for segment in segments:
    print(f"[{segment.start:.2f}s -> {segment.end:.2f}s] {segment.text}")


Langue détectée : fr
[0.00s -> 7.20s] Vous devez donner suite à tous vos engagements et nous vous les rappellerons jusqu'au dernier jour de cette législature.


In [2]:
from faster_whisper import WhisperModel

# Chemin vers le modèle CTranslate2
model_path = "./models/whisper-large-v3-french-distil-dec16-ct2/ctranslate2"

# Instanciation du modèle sur CPU avec quantification int8
model = WhisperModel(
    model_path,
    device="cpu",        # CPU uniquement
    compute_type="int8"  # accélération quantifiée
)

# Transcription d'un fichier audio
audio_file = "macron.mp3"
segments, info = model.transcribe(
    audio_file,
    beam_size=5,
    language="fr",
    condition_on_previous_text=False
)

# Affichage de la langue détectée
print("Langue détectée :", info.language)

# Affichage du texte segment par segment
for segment in segments:
    print(f"[{segment.start:.2f}s -> {segment.end:.2f}s] {segment.text}")


Langue détectée : fr
[0.00s -> 30.00s] Il y a 5 ans, l'humanité a traversé l'une des plus grandes épreuves sanitaires de notre siècle. Face à la pandémie de Covid dixneuf, ensemble, nous avons tenu bon, ensemble nous avons inventé et partagé des solutions, notamment des vaccins, et ensemble nous avons inventé de nouvelles règles, que ce soit sur le partage de la propriété intellectuelle ou pour accélérer les processus industriels. Nous avons ainsi démontré que le multilatéralisme, loin d'être un concept vain.
[30.00s -> 60.00s] Était la méthode la plus efficace pour protéger nos populations. Aujourd'hui encore, cette unité nous guide face aux menaces sanitaires nouvelles. Je pense au Mpox, au choléra, à toutes les guerres et conflits, à toutes les victimes et à ceux qui luttent chaque jour pour les soigner, et je veux dire ici notre solidarité, notre reconnaissance pour l'ensemble des soignants, des humanitaires qui sont mobilisés dans ces combats. La présentation aujourd'hui de l'acco

# Float 32

In [4]:
from faster_whisper import WhisperModel

# Chemin vers le modèle CTranslate2
model_path = "./models/whisper-large-v3-french-distil-dec16-ct2/ctranslate2"

# Instanciation du modèle sur CPU avec quantification int8
model = WhisperModel(
    model_path,
    device="cpu",        # CPU uniquement
    compute_type="float32"  # accélération quantifiée
)

# Transcription d'un fichier audio
audio_file = "test_20201111_17_21_20_8.wav"
segments, info = model.transcribe(
    audio_file,
    beam_size=5,
    language="fr",
    condition_on_previous_text=False
)

# Affichage de la langue détectée
print("Langue détectée :", info.language)

# Affichage du texte segment par segment
for segment in segments:
    print(f"[{segment.start:.2f}s -> {segment.end:.2f}s] {segment.text}")


Langue détectée : fr
[0.00s -> 7.14s] Vous devez donner suite à tous vos engagements et nous vous les rappellerons jusqu'au dernier jour de cette législature.


# Whisper large french distilled 16

# Int8

In [2]:
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq
import torch
import soundfile as sf
import librosa

# Chemin du modèle
model_name_or_path = "./models/whisper-large-v3-french-distil-dec16"

# Charger le processeur
processor = AutoProcessor.from_pretrained(model_name_or_path)

# Charger le modèle en float32
model = AutoModelForSpeechSeq2Seq.from_pretrained(model_name_or_path)
model.eval()  # mode évaluation

# Quantization dynamique en int8 pour CPU
model_int8 = torch.quantization.quantize_dynamic(
    model,                # modèle original
    {torch.nn.Linear},    # couches à quantifier
    dtype=torch.qint8
)

# Charger l'audio
audio_file = "test.wav"
audio, sr = sf.read(audio_file)
if sr != 16000:
    audio = librosa.resample(audio, orig_sr=sr, target_sr=16000)

# Préparer les entrées
inputs = processor(audio, sampling_rate=16000, return_tensors="pt")
input_features = inputs["input_features"]  # Pas d'attention_mask nécessaire

# Transcription
with torch.no_grad():
    generated_ids = model_int8.generate(input_features)
    transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

print("Transcription :", transcription)


/tmp/ipykernel_2107465/1281350102.py:17: DeprecationWarning: torch.ao.quantization is deprecated and will be removed in 2.10. 
For migrations of users: 
1. Eager mode quantization (torch.ao.quantization.quantize, torch.ao.quantization.quantize_dynamic), please migrate to use torchao eager mode quantize_ API instead 
2. FX graph mode quantization (torch.ao.quantization.quantize_fx.prepare_fx,torch.ao.quantization.quantize_fx.convert_fx, please migrate to use torchao pt2e quantization API instead (prepare_pt2e, convert_pt2e) 
3. pt2e quantization has been migrated to torchao (https://github.com/pytorch/ao/tree/main/torchao/quantization/pt2e) 
see https://github.com/pytorch/ao/issues/2259 for more details
  model_int8 = torch.quantization.quantize_dynamic(


Transcription : Vous devez donner suite à tous vos engagements et nous vous les rappellerons jusqu'au dernier jour de cette législature.


# Whisper large french

In [3]:
from huggingface_hub import snapshot_download

# Téléchargement du modèle complet
snapshot_download(
    repo_id="bofenghuang/whisper-large-v3-french",
    local_dir="./models/whisper-large-v3-french",
    allow_patterns=["*"]  # "*" pour tout télécharger
)


Fetching 22 files:   0%|          | 0/22 [00:00<?, ?it/s]

config.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

ctranslate2/model.bin:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

...

original_model.pt:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

'/home/projet9/projet9/notebooks/models/whisper-large-v3-french'

In [4]:
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq
import torch
import soundfile as sf
import librosa

# Chemin vers le modèle non distillé
model_name_or_path = "./models/whisper-large-v3-french"

# Charger le processeur
processor = AutoProcessor.from_pretrained(model_name_or_path)

# Charger le modèle en float32 et passer en mode évaluation
model = AutoModelForSpeechSeq2Seq.from_pretrained(model_name_or_path)
model.eval()

# Quantization dynamique en int8 sur CPU
model_int8 = torch.quantization.quantize_dynamic(
    model,
    {torch.nn.Linear},  # seules les couches Linear sont quantifiées
    dtype=torch.qint8
)

# Chemin vers le fichier audio
audio_file = "test.wav"

# Charger l'audio
audio, sr = sf.read(audio_file)
if sr != 16000:
    audio = librosa.resample(audio, orig_sr=sr, target_sr=16000)

# Préparer les entrées (attention_mask non nécessaire pour Whisper)
inputs = processor(audio, sampling_rate=16000, return_tensors="pt")
input_features = inputs["input_features"]

# Génération de la transcription
with torch.no_grad():
    generated_ids = model_int8.generate(input_features)
    transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

print("Transcription :", transcription)


/tmp/ipykernel_2107465/262084083.py:17: DeprecationWarning: torch.ao.quantization is deprecated and will be removed in 2.10. 
For migrations of users: 
1. Eager mode quantization (torch.ao.quantization.quantize, torch.ao.quantization.quantize_dynamic), please migrate to use torchao eager mode quantize_ API instead 
2. FX graph mode quantization (torch.ao.quantization.quantize_fx.prepare_fx,torch.ao.quantization.quantize_fx.convert_fx, please migrate to use torchao pt2e quantization API instead (prepare_pt2e, convert_pt2e) 
3. pt2e quantization has been migrated to torchao (https://github.com/pytorch/ao/tree/main/torchao/quantization/pt2e) 
see https://github.com/pytorch/ao/issues/2259 for more details
  model_int8 = torch.quantization.quantize_dynamic(


Transcription : Vous devez donner suite à tous vos engagements et nous vous les rappellerons jusqu'au dernier jour de cette législature.


# Benchmark

In [1]:
import time
import torch
import soundfile as sf
import librosa
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq
from faster_whisper import WhisperModel

# Chemin vers le fichier audio
audio_file = "test.wav"

# Charger l'audio
audio, sr = sf.read(audio_file)
if sr != 16000:
    audio = librosa.resample(audio, orig_sr=sr, target_sr=16000)

########################################
# 1. Modèle non distillé (float32 -> dyn int8)
########################################
print("\n=== Whisper Large V3 French (non distillé, dyn int8) ===")
model_name = "./models/whisper-large-v3-french"
processor = AutoProcessor.from_pretrained(model_name)
model = AutoModelForSpeechSeq2Seq.from_pretrained(model_name)
model.eval()
model_int8 = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)

inputs = processor(audio, sampling_rate=16000, return_tensors="pt")
input_features = inputs["input_features"]

start_time = time.time()
with torch.no_grad():
    generated_ids = model_int8.generate(input_features)
    transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
end_time = time.time()

print("Transcription :", transcription)
print(f"Inference time: {end_time - start_time:.2f} s")

########################################
# 2. Modèle distillé (dyn int8)
########################################
print("\n=== Whisper Large V3 French Distilled (dyn int8) ===")
model_name = "./models/whisper-large-v3-french-distil-dec16"
processor = AutoProcessor.from_pretrained(model_name)
model = AutoModelForSpeechSeq2Seq.from_pretrained(model_name)
model.eval()
model_int8 = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)

inputs = processor(audio, sampling_rate=16000, return_tensors="pt")
input_features = inputs["input_features"]

start_time = time.time()
with torch.no_grad():
    generated_ids = model_int8.generate(input_features)
    transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
end_time = time.time()

print("Transcription :", transcription)
print(f"Inference time: {end_time - start_time:.2f} s")

########################################
# 3. Modèle distillé CTranslate2 (int8)
########################################
print("\n=== Whisper Large V3 French Distilled CTranslate2 (int8) ===")
ct2_model_path = "./models/whisper-large-v3-french-distil-dec16-ct2/ctranslate2"
model = WhisperModel(
    ct2_model_path,
    device="cpu",
    compute_type="int8"
)

start_time = time.time()
segments, info = model.transcribe(
    audio_file,
    beam_size=5,
    language="fr",
    condition_on_previous_text=False
)
end_time = time.time()

print("Langue détectée :", info.language)
for segment in segments:
    print(f"[{segment.start:.2f}s -> {segment.end:.2f}s] {segment.text}")
print(f"Inference time: {end_time - start_time:.2f} s")



=== Whisper Large V3 French (non distillé, dyn int8) ===


/tmp/ipykernel_2112764/1941055802.py:24: DeprecationWarning: torch.ao.quantization is deprecated and will be removed in 2.10. 
For migrations of users: 
1. Eager mode quantization (torch.ao.quantization.quantize, torch.ao.quantization.quantize_dynamic), please migrate to use torchao eager mode quantize_ API instead 
2. FX graph mode quantization (torch.ao.quantization.quantize_fx.prepare_fx,torch.ao.quantization.quantize_fx.convert_fx, please migrate to use torchao pt2e quantization API instead (prepare_pt2e, convert_pt2e) 
3. pt2e quantization has been migrated to torchao (https://github.com/pytorch/ao/tree/main/torchao/quantization/pt2e) 
see https://github.com/pytorch/ao/issues/2259 for more details
  model_int8 = torch.quantization.quantize_dynamic(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Transcription : Vous devez donner suite à tous vos engagements et nous vous les rappellerons jusqu'au dernier jour de cette législature.
Inference time: 5.18 s

=== Whisper Large V3 French Distilled (dyn int8) ===


/tmp/ipykernel_2112764/1941055802.py:48: DeprecationWarning: torch.ao.quantization is deprecated and will be removed in 2.10. 
For migrations of users: 
1. Eager mode quantization (torch.ao.quantization.quantize, torch.ao.quantization.quantize_dynamic), please migrate to use torchao eager mode quantize_ API instead 
2. FX graph mode quantization (torch.ao.quantization.quantize_fx.prepare_fx,torch.ao.quantization.quantize_fx.convert_fx, please migrate to use torchao pt2e quantization API instead (prepare_pt2e, convert_pt2e) 
3. pt2e quantization has been migrated to torchao (https://github.com/pytorch/ao/tree/main/torchao/quantization/pt2e) 
see https://github.com/pytorch/ao/issues/2259 for more details
  model_int8 = torch.quantization.quantize_dynamic(


Transcription : Vous devez donner suite à tous vos engagements et nous vous les rappellerons jusqu'au dernier jour de cette législature.
Inference time: 3.21 s

=== Whisper Large V3 French Distilled CTranslate2 (int8) ===
Langue détectée : fr
[0.00s -> 7.20s] Vous devez donner suite à tous vos engagements et nous vous les rappellerons jusqu'au dernier jour de cette législature.
Inference time: 0.23 s


# Benchmark temporel 10 fichiers OK

In [1]:
import time
import torch
import soundfile as sf
import librosa
import json
import os
import random
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq
from faster_whisper import WhisperModel

# === Nouvelle fonction : choisir 10 fichiers audio au hasard ===
def get_random_voxpopuli_audios(base_dir="data/voxpopuli_fr_test/test_part_0", n=10):
    all_files = [f for f in os.listdir(base_dir) if f.endswith(".wav")]
    return random.sample(all_files, min(n, len(all_files)))

# Exemple : récupérer une liste de 10 fichiers
base_dir = "data/voxpopuli_fr_test/test_part_0"
random_audios = get_random_voxpopuli_audios(base_dir, n=10)
print("\n=== 10 fichiers choisis au hasard ===")
for f in random_audios:
    print(f)

# Dictionnaire pour stocker les résultats
results = {}

########################################
# Charger les modèles une seule fois
########################################
# 1. Modèle non distillé (float32 -> dyn int8)
model_name = "./models/whisper-large-v3-french"
processor_non = AutoProcessor.from_pretrained(model_name)
model_non = AutoModelForSpeechSeq2Seq.from_pretrained(model_name)
model_non.eval()
model_non_int8 = torch.quantization.quantize_dynamic(
    model_non, {torch.nn.Linear}, dtype=torch.qint8
)

# 2. Modèle distillé (dyn int8)
model_name = "./models/whisper-large-v3-french-distil-dec16"
processor_distil = AutoProcessor.from_pretrained(model_name)
model_distil = AutoModelForSpeechSeq2Seq.from_pretrained(model_name)
model_distil.eval()
model_distil_int8 = torch.quantization.quantize_dynamic(
    model_distil, {torch.nn.Linear}, dtype=torch.qint8
)

# 3. Modèle distillé CTranslate2 (int8)
ct2_model_path = "./models/whisper-large-v3-french-distil-dec16-ct2/ctranslate2"
model_ct2 = WhisperModel(
    ct2_model_path,
    device="cpu",
    compute_type="int8"
)

########################################
# Boucle sur les 10 fichiers audio
########################################
for wav_file in random_audios:
    audio_path = os.path.join(base_dir, wav_file)

    # Charger l'audio
    audio, sr = sf.read(audio_path)
    if sr != 16000:
        audio = librosa.resample(audio, orig_sr=sr, target_sr=16000)

    file_results = {}

    # ====== Modèle non distillé ======
    inputs = processor_non(audio, sampling_rate=16000, return_tensors="pt")
    input_features = inputs["input_features"]

    start_time = time.time()
    with torch.no_grad():
        generated_ids = model_non_int8.generate(input_features)
        transcription = processor_non.batch_decode(generated_ids, skip_special_tokens=True)[0]
    end_time = time.time()

    file_results["whisper_large_non_distilled"] = {
        "transcription": transcription,
        "inference_time_s": end_time - start_time
    }

    # ====== Modèle distillé ======
    inputs = processor_distil(audio, sampling_rate=16000, return_tensors="pt")
    input_features = inputs["input_features"]

    start_time = time.time()
    with torch.no_grad():
        generated_ids = model_distil_int8.generate(input_features)
        transcription = processor_distil.batch_decode(generated_ids, skip_special_tokens=True)[0]
    end_time = time.time()

    file_results["whisper_large_distilled"] = {
        "transcription": transcription,
        "inference_time_s": end_time - start_time
    }

    # ====== Modèle distillé CTranslate2 ======
    start_time = time.time()
    segments, info = model_ct2.transcribe(
        audio_path,
        beam_size=5,
        language="fr",
        condition_on_previous_text=False
    )
    end_time = time.time()

    ct2_transcription = " ".join([s.text for s in segments])
    file_results["whisper_large_distilled_ct2"] = {
        "transcription": ct2_transcription,
        "language_detected": info.language,
        "inference_time_s": end_time - start_time
    }

    # Sauvegarder les résultats pour ce fichier
    results[wav_file] = file_results

# Sauvegarder le JSON récapitulatif
with open("benchmark_results.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

print("\n? JSON récapitulatif sauvegardé dans 'benchmark_results.json'")



=== 10 fichiers choisis au hasard ===
20170426-0900-PLENARY-14-fr_20170426-19:50:54_8.wav
20141021-0900-PLENARY-10-fr_20141021-16:11:33_34.wav
20090203-0900-PLENARY-4-fr_20090203-11:21:16_9.wav
20170705-0900-PLENARY-17-fr_20170705-19:39:56_7.wav
20131121-0900-PLENARY-16-fr_20131121-15:42:54_7.wav
20090203-0900-PLENARY-4-fr_20090203-11:21:16_3.wav
20180313-0900-PLENARY-4-fr_20180313-11:13:14_4.wav
20131024-0900-PLENARY-3-fr_20131024-08:30:20_29.wav
20180417-0900-PLENARY-4-fr_20180417-11:21:18_10.wav
20170215-0900-PLENARY-16-fr_20170215-19:51:40_3.wav


/tmp/ipykernel_2125024/206810654.py:34: DeprecationWarning: torch.ao.quantization is deprecated and will be removed in 2.10. 
For migrations of users: 
1. Eager mode quantization (torch.ao.quantization.quantize, torch.ao.quantization.quantize_dynamic), please migrate to use torchao eager mode quantize_ API instead 
2. FX graph mode quantization (torch.ao.quantization.quantize_fx.prepare_fx,torch.ao.quantization.quantize_fx.convert_fx, please migrate to use torchao pt2e quantization API instead (prepare_pt2e, convert_pt2e) 
3. pt2e quantization has been migrated to torchao (https://github.com/pytorch/ao/tree/main/torchao/quantization/pt2e) 
see https://github.com/pytorch/ao/issues/2259 for more details
  model_non_int8 = torch.quantization.quantize_dynamic(
/tmp/ipykernel_2125024/206810654.py:43: DeprecationWarning: torch.ao.quantization is deprecated and will be removed in 2.10. 
For migrations of users: 
1. Eager mode quantization (torch.ao.quantization.quantize, torch.ao.quantization


? JSON récapitulatif sauvegardé dans 'benchmark_results.json'


# Benchmark temporel + precision Large / Large distilled / Large distilled CT2

In [1]:
import time
import torch
import soundfile as sf
import librosa
import json
import os
import random
import pandas as pd
import numpy as np
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq
from faster_whisper import WhisperModel
from jiwer import wer, cer

# === Nouvelle fonction : choisir n fichiers audio au hasard ===
def get_random_voxpopuli_audios(base_dir="data/voxpopuli_fr_test/test_part_0", n=100):
    all_files = [f for f in os.listdir(base_dir) if f.endswith(".wav")]
    return random.sample(all_files, min(n, len(all_files)))

# Charger le mapping id -> raw_text depuis le TSV
tsv_path = "data/voxpopuli_fr_test/asr_test.tsv"
df = pd.read_csv(tsv_path, sep="\t")
id2raw = dict(zip(df["id"], df["raw_text"]))

# Exemple : récupérer une liste
base_dir = "data/voxpopuli_fr_test/test_part_0"
random_audios = get_random_voxpopuli_audios(base_dir, n=100)

results = {}

########################################
# Charger les modèles une seule fois
########################################
# 1. Modèle non distillé
model_name = "./models/whisper-large-v3-french"
processor_non = AutoProcessor.from_pretrained(model_name)
model_non = AutoModelForSpeechSeq2Seq.from_pretrained(model_name)
model_non.eval()
model_non_int8 = torch.quantization.quantize_dynamic(
    model_non, {torch.nn.Linear}, dtype=torch.qint8
)

# 2. Modèle distillé
model_name = "./models/whisper-large-v3-french-distil-dec16"
processor_distil = AutoProcessor.from_pretrained(model_name)
model_distil = AutoModelForSpeechSeq2Seq.from_pretrained(model_name)
model_distil.eval()
model_distil_int8 = torch.quantization.quantize_dynamic(
    model_distil, {torch.nn.Linear}, dtype=torch.qint8
)

# 3. Modèle distillé CTranslate2
ct2_model_path = "./models/whisper-large-v3-french-distil-dec16-ct2/ctranslate2"
model_ct2 = WhisperModel(
    ct2_model_path,
    device="cpu",
    compute_type="int8"
)

########################################
# Boucle sur les fichiers audio
########################################
for wav_file in random_audios:
    audio_path = os.path.join(base_dir, wav_file)

    # Charger l'audio
    audio, sr = sf.read(audio_path)
    if sr != 16000:
        audio = librosa.resample(audio, orig_sr=sr, target_sr=16000)

    # Calcul de la durée de l'audio (secondes)
    duration_s = len(audio) / 16000.0

    # Récupérer la référence (raw_text)
    file_id = wav_file.replace(".wav", "")
    raw_text = id2raw.get(file_id, "")

    # Stocker au même niveau que duration_s
    file_results = {
        "duration_s": duration_s,
        "raw_text": raw_text
    }

    # ====== Modèle non distillé ======
    global_start = time.time()
    inputs = processor_non(audio, sampling_rate=16000, return_tensors="pt")
    input_features = inputs["input_features"]

    inf_start = time.time()
    with torch.no_grad():
        generated_ids = model_non_int8.generate(input_features)
        transcription = processor_non.batch_decode(generated_ids, skip_special_tokens=True)[0]
    inf_end = time.time()
    global_end = time.time()

    inference_time = inf_end - inf_start
    elapsed_time = global_end - global_start
    rtf = inference_time / duration_s
    file_results["whisper_large_non_distilled"] = {
        "transcription": transcription,
        "inference_time_s": inference_time,
        "elapsed_time_s": elapsed_time,
        "real_time_factor": rtf
    }

    # ====== Modèle distillé ======
    global_start = time.time()
    inputs = processor_distil(audio, sampling_rate=16000, return_tensors="pt")
    input_features = inputs["input_features"]

    inf_start = time.time()
    with torch.no_grad():
        generated_ids = model_distil_int8.generate(input_features)
        transcription = processor_distil.batch_decode(generated_ids, skip_special_tokens=True)[0]
    inf_end = time.time()
    global_end = time.time()

    inference_time = inf_end - inf_start
    elapsed_time = global_end - global_start
    rtf = inference_time / duration_s
    file_results["whisper_large_distilled"] = {
        "transcription": transcription,
        "inference_time_s": inference_time,
        "elapsed_time_s": elapsed_time,
        "real_time_factor": rtf
    }

    # ====== Modèle distillé CTranslate2 ======
    global_start = time.time()
    inf_start = time.time()
    segments, info = model_ct2.transcribe(
        audio_path,
        beam_size=5,
        language="fr",
        condition_on_previous_text=False
    )
    inf_end = time.time()
    global_end = time.time()

    ct2_transcription = " ".join([s.text for s in segments])
    inference_time = inf_end - inf_start
    elapsed_time = global_end - global_start
    rtf = inference_time / duration_s
    file_results["whisper_large_distilled_ct2"] = {
        "transcription": ct2_transcription,
        "language_detected": info.language,
        "inference_time_s": inference_time,
        "elapsed_time_s": elapsed_time,
        "real_time_factor": rtf
    }

    # Sauvegarder les résultats
    results[wav_file] = file_results

########################################
# Calcul du WER, CER, temps moyen, RTF
########################################
report = {}
for model_key in ["whisper_large_non_distilled", "whisper_large_distilled", "whisper_large_distilled_ct2"]:
    wers, cers, times, elapsed_times, rtfs = [], [], [], [], []
    for wav_file, res in results.items():
        file_id = wav_file.replace(".wav", "")
        if file_id not in id2raw:
            continue
        ref = str(id2raw[file_id]).lower().strip()
        hyp = str(res[model_key]["transcription"]).lower().strip()
        wers.append(wer(ref, hyp))
        cers.append(cer(ref, hyp))
        times.append(res[model_key]["inference_time_s"])
        elapsed_times.append(res[model_key]["elapsed_time_s"])
        rtfs.append(res[model_key]["real_time_factor"])
    report[model_key] = {
        "WER": float(np.mean(wers)) if wers else None,
        "CER": float(np.mean(cers)) if cers else None,
        "avg_inference_time_s": float(np.mean(times)) if times else None,
        "avg_elapsed_time_s": float(np.mean(elapsed_times)) if elapsed_times else None,
        "sum_elapsed_time_s": float(np.sum(elapsed_times)) if elapsed_times else None,  # ? Somme ajoutée
        "avg_real_time_factor": float(np.mean(rtfs)) if rtfs else None
    }

# Sauvegarde
final_output = {"results": results, "report": report}

with open("benchmark_results.json", "w", encoding="utf-8") as f:
    json.dump(final_output, f, ensure_ascii=False, indent=4)

print("\n? JSON enrichi avec inference_time, elapsed_time, RTF, WER, CER et somme totale des elapsed_time")



/tmp/ipykernel_2460959/1897776081.py:38: DeprecationWarning: torch.ao.quantization is deprecated and will be removed in 2.10. 
For migrations of users: 
1. Eager mode quantization (torch.ao.quantization.quantize, torch.ao.quantization.quantize_dynamic), please migrate to use torchao eager mode quantize_ API instead 
2. FX graph mode quantization (torch.ao.quantization.quantize_fx.prepare_fx,torch.ao.quantization.quantize_fx.convert_fx, please migrate to use torchao pt2e quantization API instead (prepare_pt2e, convert_pt2e) 
3. pt2e quantization has been migrated to torchao (https://github.com/pytorch/ao/tree/main/torchao/quantization/pt2e) 
see https://github.com/pytorch/ao/issues/2259 for more details
  model_non_int8 = torch.quantization.quantize_dynamic(
/tmp/ipykernel_2460959/1897776081.py:47: DeprecationWarning: torch.ao.quantization is deprecated and will be removed in 2.10. 
For migrations of users: 
1. Eager mode quantization (torch.ao.quantization.quantize, torch.ao.quantizati


? JSON enrichi avec inference_time, elapsed_time, RTF, WER, CER et somme totale des elapsed_time
